In [ ]:
import json
import logging
import matplotlib.pyplot as plt
import requests
import base64, json, time
import os
import unicodedata

from tqdm import tqdm
from pathlib import Path
from openai import OpenAI
from PIL import Image

from AgentFactory import AgentFactory
from src.utils import clear_json

logging.basicConfig(level=logging.INFO)

def load_image(image_path: str) -> str:
    """
    读取一张图片，返回 base64 编码字符串。

    由于 Ubuntu 文件系统上文件名可能以 NFD（拆分）形式存储，
    本函数会依次尝试 NFC（预组合）和 NFD（拆分）两种形式。
    """
    # 先把路径包装成 Path 对象
    p = Path(image_path)
    # 两种规范化形式
    candidates = [
        # 原样
        str(p),
        # NFC：预组合（é）
        unicodedata.normalize('NFC', str(p)),
        # NFD：拆分（e + ́）
        unicodedata.normalize('NFD', str(p)),
    ]

    last_err = None
    for path_str in candidates:
        try:
            with open(path_str, "rb") as f:
                data = f.read()
            # 成功读取后立即编码并返回
            return base64.b64encode(data).decode("utf-8")
        except FileNotFoundError as e:
            last_err = e
            # 如果这个形式不存在，就继续下一个
            continue

    # 如果所有形式都无法打开，就抛出最后一个错误
    raise last_err

/home/youyuan/miniconda3/envs/Iris/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def collect_image_metadata(root_dir: str = "/home/youyuan/Transcreation/Iris/download_images/image-transcreation/outputs/part1/zhang", source_dir: str = "/home/youyuan/Transcreation/Iris/download_images/image-transcreation/part1"):
    """
    遍历 root_dir 目录，收集所有图片及其相关元信息。
    假设目录结构：
    root_dir/
      ├── country_A/
      │     ├── Step1X/
      │     └── Gemini/
      ├── country_B/
      │     ├── Step1X/
      │     └── Gemini/
      ...
    图片文件名格式：<source>_<category>_<object>.<ext>
    """
    root = Path(root_dir)
    images = []

    for country_folder in root.iterdir():
        if not country_folder.is_dir():
            continue
        target_country = country_folder.name
        for step_folder in ["Step1X", "Gemini"]:
            folder = country_folder / step_folder
            if not folder.exists():
                continue
            for img_path in folder.iterdir():
                if not img_path.is_file():
                    continue
                # 只处理常见图片格式
                if img_path.suffix.lower() not in {".jpg", ".jpeg", ".png", ".bmp", ".gif"}:
                    continue

                # 解析文件名：source_category_object.ext
                name_parts = img_path.stem.split("_")
                if len(name_parts) < 3:
                    # 如果不符合预期格式，可以跳过或自行处理
                    continue
                source_country = name_parts[0]
                category = name_parts[1]
                # 剩下部分作为 object 名称（以防 object 名含下划线）
                object_name = "_".join(name_parts[2:])

                images.append({
                    "file_path": str(img_path),
                    "source_path": f"{source_dir}/{source_country}/{category}_{object_name}{img_path.suffix.lower()}",
                    "target_country": target_country,
                    "step": step_folder,
                    "source_country": source_country,
                    "category": category,
                    "object_name": object_name,
                })

    return images

We define the TranscreationEvaluator class, following [S Khanuja's approach](https://arxiv.org/abs/2412.13717) to utilize a LLM for evaluating transcreation. 

In [3]:
class TranscreationEvaluator:
    def __init__(self, config):
        self.model_name = config["model_name"]
        self.api_key = config["api_key"]
        self.client = OpenAI(
            api_key=self.api_key,
        )

    def download_batch_results(self, file_id: str, save_path: str):
        """
        通过 REST API 下载 batch 输出文件内容并保存到本地
        """
        url = f"https://api.openai.com/v1/files/{file_id}/content"
        headers = {
            "Authorization": f"Bearer {self.api_key}"
        }
        resp = requests.get(url, headers=headers, stream=True)
        resp.raise_for_status()
    
        with open(save_path, "wb") as f:
            for chunk in resp.iter_content(chunk_size=8_192):
                f.write(chunk)

    def get_bodies(self, img1_path, img2_path, target_country, category = None):
        
        category_patch = f" and its category '{category}'"
        try:
            base64_image_1 = load_image(img1_path)
            base64_image_2 = load_image(img2_path)
        except Exception as e:
            logging.error(f"Error loading images: {e}")
            return {}

        instructions = {
            "culture-relevance": f"You are an expert in evaluating the cultural relevance of images.",
            "semantic": f"You are an expert in judging the semantic equality between images.",
            "visual": "You are an expert in judging the visual similarity between images."
        }

        prompt = {   
            "culture-relevance":[
                {"type": "text", "text": f"You will be given two images. You have to assess how culturally relevant both images are with respect to the culture of {target_country}."},
                {"type": "text", "text": f"Please explain your reasoning step by step for both images, specifically considering cultural symbols, styles, traditions, or any features that align with the culture of the speaking population of {target_country}.\n"},
                {"type": "text", "text": f"For each image, the final score should be a number between 1 to 5, where 1 and 5 mean the following:\n"},
                {"type": "text", "text": f"1 = Not culturally relevant,\n"},
                {"type": "text", "text": f"5 = Culturally relevant.\n"},
                {"type": "image_url", "image_url": { "url": f"data:image/jpeg;base64,{base64_image_1}"}},
                {"type": "image_url", "image_url": { "url": f"data:image/jpeg;base64,{base64_image_2}"}},
                {"type": "text", "text": f"The output should be a JSON object ONLY with the following format:\n"},
                {"type": "text", "text": "{\"first_reasoning\": ..., \"first_score\": number, \"second_reasoning\": ..., \"second_score\": number}\n"},
            ],
            "semantic":[
                {"type": "text", "text": f"Given the input image{category_patch}, determine if the comparison image belongs to the same category as the input.\n"},
                {"type": "text", "text": f"Please explain your reasoning step by step, and provide a final score at the end.\n"},
                {"type": "text", "text": f"The score should be between 1 and 5, where:\n"},
                {"type": "text", "text": f"1 = Dissimilar Category,\n"},
                {"type": "text", "text": f"5 = Same Category.\n"},
                {"type": "image_url", "image_url": { "url": f"data:image/jpeg;base64,{base64_image_1}"}},
                {"type": "image_url", "image_url": { "url": f"data:image/jpeg;base64,{base64_image_2}"}},
                {"type": "text", "text": f"The output should be a JSON object ONLY with the following format:\n"},
                {"type": "text", "text": "{\"reasoning\": ..., \"score\": number}\n"},
            ],
            "visual":[
                {"type": "text", "text": f"Given the input image, determine if there are visual changes in the second image as compared to the first."},
                {"type": "text", "text": f"Please explain your reasoning step by step, and provide a final score at the end."},
                {"type": "text", "text": f"The score should be between 1 and 5, where:\n1 = No visual change,\n5 = High visual changes."},
                {"type": "image_url", "image_url": { "url": f"data:image/jpeg;base64,{base64_image_1}"}},
                {"type": "image_url", "image_url": { "url": f"data:image/jpeg;base64,{base64_image_2}"}},
                {"type": "text", "text": "The output should be a JSON object ONLY with the following STRICT format: {\"reasoning\": ..., \"score\": number}"},
            ],
        }

        bodies = {}
        for key in ["culture-relevance", "semantic", "visual"]:

            history = [
                {
                    "role": "assistant", 
                    "content": [
                        { "type": "text", "text": instructions[key]}
                    ]
                },
                {
                    "role": "user", 
                    "content": prompt[key]
                }
            ]
                
            bodies[key] = {
                "model": self.model_name,
                "temperature": 0.0,
                "messages": history
            }
        
        return bodies

    def save_task_file(self, images, batch_file="batch.jsonl", record_file="batch_record.json"):
        tasks = []
        record = {}
        for item in tqdm(images):
            img1_path = item["source_path"]
            img2_path = item["file_path"]
            target_country = item["target_country"]
            category = item["category"]

            bodies = self.get_bodies(img1_path, img2_path, target_country, category)
            for key, body in bodies.items():
                custom_id = f"{item['source_country']}_{item['target_country']}_{item['category']}_{item['object_name']}_{key}"
                tasks.append({
                    "custom_id": custom_id,
                    "method": "POST",
                    "url": "/v1/chat/completions",
                    "body": body
                })
                record[custom_id] = item
        
        with open(batch_file, "w", encoding="utf-8") as f:
            for t in tasks:
                f.write(json.dumps(t, ensure_ascii=False) + "\n")
        with open(record_file, "w", encoding="utf-8") as f:
            f.write(json.dumps(record, ensure_ascii=False, indent=4))
            
        print(f"Batch 作业文件已创建，包含 {len(tasks)} 个任务。")

    def sumbit_tasks(self, batch_file="batch.jsonl"):

        upload_resp = self.client.files.create(
            file=open(batch_file, "rb"),
            purpose="batch"
        )
        file_id = upload_resp.id

        batch_job = self.client.batches.create(
            input_file_id=file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h"
        )
        batch_id = batch_job.id
        print(f"Batch 作业已创建，file ID: {file_id}, 作业 ID: {batch_id}")

        return batch_job

    def delete_batch(self, batch_id):
        """
        删除指定的 Batch 作业。
        """
        try:
            self.client.files.delete(batch_id)
            print(f"Batch 作业 {batch_id} 已删除。")
        except Exception as e:
            print(f"删除 Batch 作业 {batch_id} 时出错：{e}")

    def query_results(self, batch_id, results_file=None):

        while True:
            status = self.client.batches.retrieve(batch_id)
            if status.status in ("completed", "failed"):
                break
            print(f"当前状态：{status.status}，等待中…")
            time.sleep(10)

        if status.status == "failed":
            print("Batch 作业验证或执行失败，错误信息：", status.errors)
            return status

        if results_file:

            results_file_id = status.output_file_id if status.output_file_id else status.error_file_id
            self.download_batch_results(results_file_id, results_file)
            print(f"处理完成，结果保存在{results_file}")

        return status
    
    def analysis(self, results_path):

        return
        for k,v in prompt.items():
            self.agent.reset_msg_history()
            try:
                response, text = self.agent.get_response(v, images = images)
                text = clear_json(text)
                result = json.loads(text)
                answer[k] = result
            except Exception as e:
                logging.error(f"Error processing {k} evaluation: {e}")

        return answer


Loading the evaluator.

In [4]:
# config_path: str = "configs/conf_Gemini.json"
config_path: str = "configs/conf_GPT.json"
config = json.load(open(config_path, "r"))
config = config["model"]

evaluator = TranscreationEvaluator(config)

Loading the test images.

In [ ]:
all_images = collect_image_metadata()
# split images per N
N = 32
images_split = [all_images[i:i + N] for i in range(0, len(all_images), N)]
for i, images in enumerate(images_split):
    print(f"Processing batch {i + 1}/{len(images_split)} with {len(images)} images...")
    # Save each batch to a separate file
    batch_file = f"output/batch_{i + 1}.jsonl"
    record_file = f"output/batch_record_{i + 1}.json"
    evaluator.save_task_file(images, batch_file=batch_file, record_file=record_file)

In [ ]:
jobs = []
for i in tqdm(range(len(images_split))):
    batch_file = f"output/batch_{i + 1}.jsonl"
    results_file = f"output/batch_results_{i + 1}.jsonl"
    job = evaluator.sumbit_tasks(batch_file=batch_file)
    jobs.append(job)

with open("output/batch_jobs.json", "w", encoding="utf-8") as f:
    f.write(json.dumps([job.to_dict() for job in jobs], ensure_ascii=False, indent=4))

In [ ]:
tmp_jobs = {300: [], 311: [], 397: []}
for it in [300, 311, 397]:
    for batch_file in [f"output/batch_{it}_split1.jsonl", f"output/batch_{it}_split2.jsonl"]:
        job = evaluator.sumbit_tasks(batch_file=batch_file)
        tmp_jobs[it].append(job)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/batches "HTTP/1.1 200 OK"


Batch 作业已创建，file ID: file-AV1RNqji27F129SLhPK6W7, 作业 ID: batch_68556cc8b43c8190947f0bbf21777789


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/batches "HTTP/1.1 200 OK"


Batch 作业已创建，file ID: file-Pr4Lt9Nz3hi7tpTVT1wbcP, 作业 ID: batch_68556ced3334819091f137bca9db0e19


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/batches "HTTP/1.1 200 OK"


...

Batch 作业已创建，file ID: file-DmLbMk8wT4DVyo3vYXTL8v, 作业 ID: batch_68556d91e7888190a99ae107653021a8


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/batches "HTTP/1.1 200 OK"


Batch 作业已创建，file ID: file-Tj3rmksFRTTLZG42NjvN1x, 作业 ID: batch_68556da95058819097c99ff1d7ec7c2b


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/batches "HTTP/1.1 200 OK"


Batch 作业已创建，file ID: file-HTrhoaFjRKvbnuvJBPHfSs, 作业 ID: batch_68556db82fdc819096e2c16b06e17c23


In [15]:
for it, jobs in tmp_jobs.items():
    cnt = 0
    for job in jobs:
        cnt += 1
        batch_id = job.id
        results_file = f"output/batch_results_{it}_{cnt}.jsonl"
        status = evaluator.query_results(batch_id, results_file=results_file)
        print(f"Batch {batch_id} status: {status.status}")

INFO:httpx:HTTP Request: GET https://api.openai.com/v1/batches/batch_68556cc8b43c8190947f0bbf21777789 "HTTP/1.1 200 OK"


处理完成，结果保存在output/batch_results_300_1.jsonl
Batch batch_68556cc8b43c8190947f0bbf21777789 status: completed


INFO:httpx:HTTP Request: GET https://api.openai.com/v1/batches/batch_68556ced3334819091f137bca9db0e19 "HTTP/1.1 200 OK"


当前状态：in_progress，等待中…


INFO:httpx:HTTP Request: GET https://api.openai.com/v1/batches/batch_68556ced3334819091f137bca9db0e19 "HTTP/1.1 200 OK"


...

处理完成，结果保存在output/batch_results_311_2.jsonl
Batch batch_68556d91e7888190a99ae107653021a8 status: completed


INFO:httpx:HTTP Request: GET https://api.openai.com/v1/batches/batch_68556da95058819097c99ff1d7ec7c2b "HTTP/1.1 200 OK"


处理完成，结果保存在output/batch_results_397_1.jsonl
Batch batch_68556da95058819097c99ff1d7ec7c2b status: completed


INFO:httpx:HTTP Request: GET https://api.openai.com/v1/batches/batch_68556db82fdc819096e2c16b06e17c23 "HTTP/1.1 200 OK"


处理完成，结果保存在output/batch_results_397_2.jsonl
Batch batch_68556db82fdc819096e2c16b06e17c23 status: completed


In [5]:
with open("output/batch_jobs.json", "r") as f:
    jobs = json.load(f)

In [6]:
jobs[0]

{'id': 'batch_6855217a8a688190ad81bc0156174fe4',
 'input_file_id': 'file-MLM1Xrafi6o2hPncrxp2GD'}

In [ ]:
status = evaluator.query_results("batch_6855217a8a688190ad81bc0156174fe4")

In [ ]:
for k, v in status.to_dict().items():
    print(f"{k}: {v}")

In [7]:
statuses = []
for cnt, job in tqdm(enumerate(jobs, start=1), total=len(jobs)):
    # evaluator.delete_batch(job["input_file_id"])
    # if job["output_file_id"]:
    #     evaluator.delete_batch(job["output_file_id"])
    # if job["error_file_id"]:
    #     evaluator.delete_batch(job["error_file_id"])
    batch_id = job["id"]
    results_file = f"output/batch_results_{cnt}.jsonl"
    status = evaluator.query_results(batch_id, results_file=results_file)
    statuses.append(status)

  0%|          | 1/443 [00:02<16:04,  2.18s/it]

处理完成，结果保存在output/batch_results_1.jsonl


INFO:httpx:HTTP Request: GET https://api.openai.com/v1/batches/batch_685526e43b248190bd378bf2906c7ad0 "HTTP/1.1 200 OK"
  0%|          | 2/443 [00:03<10:44,  1.46s/it]

处理完成，结果保存在output/batch_results_2.jsonl


INFO:httpx:HTTP Request: GET https://api.openai.com/v1/batches/batch_685526f80e7881908cbd31cef118c964 "HTTP/1.1 200 OK"
  1%|          | 3/443 [00:03<08:37,  1.18s/it]

...

处理完成，结果保存在output/batch_results_441.jsonl


INFO:httpx:HTTP Request: GET https://api.openai.com/v1/batches/batch_68554d18b88c8190b7d70195d0950eae "HTTP/1.1 200 OK"
100%|█████████▉| 442/443 [07:33<00:01,  1.07s/it]

处理完成，结果保存在output/batch_results_442.jsonl


INFO:httpx:HTTP Request: GET https://api.openai.com/v1/batches/batch_68554d1c735c8190a76a9e1006ad22dd "HTTP/1.1 200 OK"
100%|██████████| 443/443 [07:34<00:00,  1.03s/it]

处理完成，结果保存在output/batch_results_443.jsonl
